In [171]:
import pandas as pd
import gzip

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

reviews = getDF('C:/Users/tc2777/PycharmProjects/personalization-theory/reviews_Video_Games.json.gz')
meta = getDF('C:/Users/tc2777/PycharmProjects/personalization-theory/meta_Video_Games.json.gz')

In [175]:
print(reviews.columns.tolist())
print(meta.columns.tolist())
print(meta.shape)

['reviewerName', 'summary', 'reviewTime', 'asin', 'overall', 'unixReviewTime', 'reviewText', 'reviewerID', 'helpful']
['price', 'salesRank', 'categories', 'description', 'asin', 'related', 'imUrl', 'title', 'brand', 'cat']
(50759, 10)


In [167]:
print(meta.loc[333, ['asin', 'categories']])

asin          B00000JFBR                   
categories    [[Video Games, More Systems]]
Name: 333, dtype: object


In [172]:
def assignment(item):
    wanted_categories = ['PC', 'Mac', 'More Systems', ' Nintendo', 'PlayStation', 'Sony', 'Video Games', 'Wii', 'Xbox']
    counter = 0
    for category in wanted_categories:
        if category in item:
            return wanted_categories[counter]
        else:
            counter += 1


def assignment2(item):
    wanted_categories = ['PC', 'Mac', 'More Systems', ' Nintendo', 'PlayStation', 'Sony', 'Video Games', 'Wii', 'Xbox']
    counter = 0
    category = ''
    for category2 in wanted_categories:
        if category2 in item:
            category = category + " " + category2
            counter += 1
        else:
            counter += 1
    return category.strip()


def pick_categories(row):
    each_game_category = row['categories']
    category = ''
    if len(each_game_category) > 1:
        for i in range(len(each_game_category)):
            each_game = each_game_category[i]
            category = category + " " + each_game[1]
        return assignment2(category)
    else:
        if len(each_game_category[0]) == 1:
            return assignment(each_game_category[0][0])
        else:
            return assignment(each_game_category[0][1])

In [173]:
different_cat = meta.apply(lambda row: pick_categories(row), axis=1)
meta['cat'] = different_cat
meta = meta[meta['cat']!='']


In [176]:
cat_table = pd.crosstab(index = meta['cat'], columns="count")
print(cat_table.sort_values(by='count', ascending=0))

col_0                                           count
cat                                                  
PC                                              15742
More Systems                                    11918
Xbox                                            4557 
PlayStation                                     4225 
Wii                                             4013 
Nintendo                                        2140 
Sony                                            1893 
PC Mac                                          781  
Mac                                             650  
PC More Systems                                 501  
Video Games                                     416  
Nintendo PlayStation Wii Xbox                   232  
More Systems PlayStation                        124  
More Systems Wii                                91   
More Systems Xbox                               74   
More Systems PlayStation Wii Xbox               63   
PlayStation Xbox            

col_0                                            count
cat                                                   
                                                 194  
  Nintendo                                       2140 
  Nintendo PlayStation Wii                       2    
  Nintendo PlayStation Wii Xbox                  232  
  Nintendo Sony                                  4    
  Nintendo Wii                                   21   
  Nintendo Wii Xbox                              1    
  Nintendo Xbox                                  1    
 Mac                                             213  
 Mac More Systems                                11   
 Mac PlayStation                                 1    
 Mac Sony                                        2    
 Mac Xbox                                        6    
 More Systems                                    3521 
 More Systems  Nintendo                          51   
 More Systems  Nintendo PlayStation Wii Xbox     16   
 More Syst

In [131]:
pd.set_option('display.max_colwidth', -1)
meta2['imUrl'][meta2.cat.str.contains('Utilities Kids & Family PC') == True]

8996     http://ecx.images-amazon.com/images/I/51ZZ2M61W3L._SY300_.jpg  
9156     http://ecx.images-amazon.com/images/I/51W64GN4DHL._SX300_.jpg  
9198     http://ecx.images-amazon.com/images/I/51DWC1G8AXL._SX300_.jpg  
9202     http://ecx.images-amazon.com/images/I/51FHM39GGTL._SX300_.jpg  
10840    http://ecx.images-amazon.com/images/I/51T52JJMCBL._SX300_.jpg  
16115    http://ecx.images-amazon.com/images/I/61yd%2B8iSpkL._SY300_.jpg
Name: imUrl, dtype: object